In [1]:
!git clone https://github.com/CCogS-Mx/text-preprocessing.git

fatal: destination path 'text-preprocessing' already exists and is not an empty directory.


## Imports

In [6]:
import os
import pandas as pd
import numpy as np
import pickle

# Algoritmos de ML
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import KFold, GridSearchCV


## Data

In [5]:
embeddings = [pickle.load(open('Embeddings/bert_avg_sent_embeddings_task_1_homomex.pkl', 'rb')),
              pickle.load(open('Embeddings/bert_sent_embeddings_task_1_homomex.pkl', 'rb')),
              pickle.load(open('Embeddings/fasttext_avg_sent_embeddings_task_1_homomex.pkl', 'rb')),
              pickle.load(open('Embeddings/fasttext_sent_embeddings_task_1_homomex.pkl', 'rb'))]
embeddings_names = ['BERT_avg', 'BERT', 'FastText_avg', 'FastText'] 

In [7]:
main_path = 'Data/'
training = pd.read_csv(f'{main_path}homomex_training.csv').drop('index', axis=1)

In [8]:
training.label = training.label.fillna('NR')
dictionary_list = [{'P': 0, 'NP': 1, 'NR': 2}]
training.label.replace(list(dictionary_list[0].keys()), 
            list(dictionary_list[0].values()), 
            inplace=True)


In [9]:
y_train = training.label.tolist()

In [17]:
len(embeddings[0]), len(y_train)

(7000, 7000)

## Finding best hyperparams

In [12]:
models = [LogisticRegression(), RandomForestClassifier(), LinearSVC(), KNeighborsClassifier()]
model_names = ['Regresión logística', 'Random Forest', 'SVM Lineal', 'KNN']

folds = 5
grid_params  = [{"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_estimators": [10, 50, 100], "max_depth":[10, 50, 100], "random_state": [42]},
                {"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_neighbors":[3], "weights":['uniform', 'distance']}]

In [18]:
for embedding in embeddings:
    print(f'Embedding: {embeddings_names[embeddings.index(embedding)]}')
    for j, model in enumerate(models):
        print(f'Model {j}: {model_names[j]}')
        kFold_cv = KFold(n_splits=folds,
                        shuffle=True, 
                        random_state=42)
        param_grid = grid_params
        grid_search = GridSearchCV(model,
                                param_grid[j],
                                cv=kFold_cv.split(embedding, y_train),
                                scoring='f1_macro',
                                verbose=3,
                                return_train_score=True)
        grid_search.fit(embedding, y_train)

        print("Best parameters: ", grid_search.best_params_)
        print("Accuracy :", grid_search.best_score_)

Embedding: BERT_avg
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.631, test=0.611) total time=   1.1s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.623, test=0.608) total time=   1.2s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.635, test=0.593) total time=   1.2s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.631, test=0.600) total time=   1.2s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.627, test=0.628) total time=   1.3s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.631, test=0.611) total time=   1.0s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.623, test=0.608) total time=   1.1s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, random_state

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END C=1.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.856, test=0.666) total time=   8.8s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END C=1.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.846, test=0.697) total time=   8.6s


...

[CV 4/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.436, test=0.432) total time=  38.9s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.523, test=0.511) total time=  37.6s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters:  {'C': 0.1, 'max_iter': 10000, 'penalty': 'l2', 'random_state': 42}
Accuracy : 0.6760195896180071
Model 3: KNN
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.759, test=0.520) total time=   0.0s
[CV 2/5] END n_neighbors=3, weights=uniform;, score=(train=0.750, test=0.540) total time=   0.0s
[CV 3/5] END n_neighbors=3, weights=uniform;, score=(train=0.754, test=0.552) total time=   0.0s
[CV 4/5] END n_neighbors=3, weights=uniform;, score=(train=0.750, test=0.541) total time=   0.0s
[CV 5/5] END n_neighbors=3, weights=uniform;, score=(train=0.754, test=0.549) total time=   0.0s
[CV 1/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.527) total time=   0.0s
[CV 2/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.541) total time=   0.0s
[CV 3/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.556) total time=   0.0s
[CV 4/5] END n_neighbors=3, weigh